# Odometry

In [3]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

## Calibration

-Results-  
speed 100, time 4s, angle 180   ! not percise measurement !  
speed 80, time 4.8s, angle 180  (uncertainty: 5°)  
  
speed 80, time 5s, distance 15,2 cm (uncertainty: 1mm)  
speed 80, time 1.04s, distance 3 cm (dist entre cases)  

In [69]:
temps = 0
MOTOR_SPEED = 80

while (1) :
    if (robot.get_var("button.right") == 1):
        t0 = time.time()
        robot.set_var("motor.left.target", int(2**16)-int(MOTOR_SPEED))
        robot.set_var("motor.right.target", int(MOTOR_SPEED))

    if (robot.get_var("button.left") == 1):
        t1 = time.time()
        temps = t1-t0
        print('time: ',temps)
        robot.set_var("motor.left.target", 0)
        robot.set_var("motor.right.target", 0)

time:  9.929829359054565
time:  9.930827140808105
time:  9.931825876235962
time:  9.931825876235962
time:  9.932821035385132
time:  9.932821035385132
time:  9.933819055557251
time:  9.933819055557251
time:  9.934816122055054
time:  9.934816122055054
time:  9.935813188552856
time:  9.93681025505066
time:  9.93681025505066
time:  9.937807083129883
time:  9.937807083129883
time:  9.93880581855774
time:  9.93880581855774
time:  9.939802885055542
time:  9.939802885055542
time:  9.940799474716187
time:  9.940799474716187
time:  9.941796779632568
time:  9.942793607711792
time:  9.942793607711792
time:  9.94379210472107
time:  9.94379210472107
time:  9.944789409637451
time:  9.944789409637451
time:  9.945788145065308
time:  9.946784496307373
time:  9.946784496307373
time:  9.947782039642334
time:  9.947782039642334
time:  9.948778629302979
time:  9.948778629302979
time:  9.94977617263794
time:  9.94977617263794
time:  9.950773000717163
time:  9.950773000717163
time:  9.951770544052124
time:  9

KeyboardInterrupt: 

In [ ]:
robot.set_var("motor.left.target", MOTOR_SPEED)
robot.set_var("motor.right.target",MOTOR_SPEED)
time.sleep(1.04)
robot.set_var("motor.left.target", 0)
robot.set_var("motor.right.target", 0)

## Calibration test

In [ ]:
FORWARD = 1
TURN = 2
ONETURN = 4.8
SAMPLING = 5
MOTOR_SPEED = 80

state = 0
temps = 0
t0 = time.time()

while (1):
    
    if (robot.get_var("button.forward") == 1):
        state = FORWARD
        t0 = time.time()
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", MOTOR_SPEED)
        
        
    if (robot.get_var("button.center") == 1):
        state = 0
        t1 = time.time()
        temps = t1-t0
        print('time: ',temps)
        robot.set_var("motor.left.target", 0)
        robot.set_var("motor.right.target", 0)
   
    t1 = time.time() 
    temps = t1-t0

    if state == FORWARD:
        if temps > ONETURN :
            state = TURN
            t0 = time.time()
            robot.set_var("motor.left.target", int(int(2**16)-MOTOR_SPEED))
            robot.set_var("motor.right.target", int(MOTOR_SPEED))
            
    t1 = time.time() 
    temps = t1-t0
    
    if state == TURN :
        if temps > ONETURN :
            state = FORWARD
            t0 = time.time()
            robot.set_var("motor.left.target", MOTOR_SPEED)
            robot.set_var("motor.right.target", MOTOR_SPEED-1)



## Position code

Assemptions :  
the 0,0 pts is in the left corner of the map  
α: angle of the robot in radient  
γ: angle between the target and the robot (the unknown)  

We are in degrees

In [77]:
CALIBRATION_TIME_ANGLE = 4.8
CALIBRATION_ANGLE = 180
MOTOR_SPEED = 80

def robot_turn(gamma, direction,alpha): #return alpha, direction
    gamma = abs(gamma)
    time_turn = CALIBRATION_TIME_ANGLE*gamma/CALIBRATION_ANGLE
    
    if direction == "right":
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", int(int(2**16)-MOTOR_SPEED))
        time.sleep(time_turn)
        #robot.set_var("motor.left.target", 0)
        #robot.set_var("motor.right.target", 0)
        
        alpha = angle_between_pi(alpha-math.radians(gamma))
        
    if direction == "left":
        robot.set_var("motor.left.target", int(int(2**16)-MOTOR_SPEED))
        robot.set_var("motor.right.target", MOTOR_SPEED)
        time.sleep(time_turn)
        #robot.set_var("motor.left.target", 0)
        #robot.set_var("motor.right.target", 0)
        
        alpha = angle_between_pi(alpha+math.radians(gamma))
    
    direction = "forward"
    return alpha, direction

In [78]:
def angle_between_pi(angle): #return angle in [-pi,pi]
    if angle > math.pi :
        angle = angle - 2*math.pi
        return angle
    elif angle < -math.pi:
        angle = angle + 2*math.pi
        return angle
    else :
        return angle

In [79]:
CALIBRATION_TIME = 1.04
DISTANCE_CALLIBRATION = 3
MOTOR_SPEED = 80

def robot_move_forward(direction, distance,xr,xt,yr,yt): # distance is in cm , returns new xr, yr
    
    if direction == "forward" :
        time_forward = CALIBRATION_TIME*distance/DISTANCE_CALLIBRATION
        
        robot.set_var("motor.left.target", MOTOR_SPEED)
        robot.set_var("motor.right.target", MOTOR_SPEED)
        time.sleep(time_forward)
        #robot.set_var("motor.left.target", 0)
        #robot.set_var("motor.right.target", 0)
        xr=xt
        yr=yt
        
    return xr, yr

In [80]:
def robot_stop():
    robot.set_var("motor.left.target", 0)
    robot.set_var("motor.right.target", 0)
    return

In [81]:
FOURCHETTE = 5 # to uniform with the code
COTE_CARRE_MAP = 3

def robot_evaluation (xr,xt,yr,yt, alpha): #return gamma in degrees, direction, distance in cm
    
    alpha = math.degrees(alpha)
    xr=xr*COTE_CARRE_MAP
    xt=xt*COTE_CARRE_MAP
    yr=yr*COTE_CARRE_MAP
    yt=yt*COTE_CARRE_MAP
    
    if yt>yr and xt<xr :
        gamma = math.degrees(math.atan2(yt-yr,xr-xt))

        if alpha<=0 :
            gamma = gamma + 180+alpha

            if gamma < 180 :
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn left
                gamma = 360-gamma
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = 180 - gamma - alpha

            if gamma<0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)


    if yt<yr and xt<xr :  
        gamma = math.degrees(math.atan2(yr-yt,xr-xt))

        if alpha>=0 :
            gamma = gamma +180-alpha

            if gamma < 180 :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn right
                gamma = 360-gamma
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = 180 - gamma + alpha

            if gamma>0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)


    if yt<yr and xt>xr :  
        gamma = math.degrees(math.atan2(yr-yt,xt-xr))

        if alpha>=0 :
            gamma = gamma + alpha

            if gamma > 180 :
                #robot turn left
                gamma = 360-gamma
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = -alpha - gamma

            if gamma<0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        

    if yt>yr and xt>xr :
        gamma = math.degrees(math.atan2(yt-yr,xt-xr))

        if alpha<=0 :
            gamma = gamma - alpha

            if gamma > 180 :
                #robot turn right
                gamma = 360-gamma
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else:
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
        else :
            gamma = alpha - gamma

            if gamma>0:
                #robot turn right
                return abs(gamma), "right", math.sqrt((yt-yr)**2 + (xr-xt)**2)
            else :
                #robot turn left
                return abs(gamma), "left", math.sqrt((yt-yr)**2 + (xr-xt)**2)
    
    
    if yt==yr and xt>xr :
        gamma = 0
        distance = xt-xr
        
        if abs(alpha)< FOURCHETTE :
            #robot move forward
            return abs(gamma), "forward", distance
        elif alpha < -(FOURCHETTE/2) :
            #robot turn left
            return abs(gamma), "left", distance
        else :
            #robot turn right
            return abs(gamma), "right", distance
            
    if yt==yr and xt<xr :
        gamma = 180
        distance = xr-xt

        if alpha < 0 :
            gamma = gamma + alpha
            #robot turn right
            return abs(gamma), "right", distance
        else:
            gamma = gamma - alpha
            #robot turn left
            return abs(gamma), "left", distance
            
    
    if yt<yr and xt==xr :
        gamma = 90                           #=-90
        gamma = gamma+alpha
        distance = yr-yt

        if gamma < 0:
            #robot turn left
            return abs(gamma), "left", distance
        elif gamma > 180 :
            #robot turn left
            gamma= 360-gamma
            return abs(gamma), "left", distance
        else:
            #robot turn right
            return abs(gamma), "right", distance
            
            
    if yt>yr and xt==xr :
        gamma = 90
        gamma = gamma-alpha
        distance = yt-yr

        if gamma < 0 :
            #robot turn right
            return abs(gamma), "right", distance
        elif gamma > 180 :
            #robot turn right
            gamma= 360-gamma
            return abs(gamma), "right", distance
        else:
            #robot turn left
            return abs(gamma), "left", distance

In [82]:
def target_not_reach(xr,xt,yr,yt): # return bool yes/no
    if yt==yr and xt==xr :
        return False
    else:
        return True

In [83]:
# We got :
target_list = [[2,2],[3,2],[4,3],[5,4],[5,5]]
robot_pos = [1,2]
alpha = 0 # in radian

for target in target_list :
    
    print('target: ', target)

    while (target_not_reach(robot_pos[0],target[0],robot_pos[1],target[1])):
        gamma, direction, distance = robot_evaluation (robot_pos[0],target[0],robot_pos[1],target[1], alpha)

        print('gamma: ',gamma,'direction: ',direction,'distance: ',distance)

        alpha, direction = robot_turn(gamma, direction, alpha)

        print('alpha: ',math.degrees(alpha),'direction: ',direction)

        robot_pos[0], robot_pos[1] = robot_move_forward(direction, distance,robot_pos[0],target[0],robot_pos[1],target[1])

        print('robot: ',robot_pos)

        robot_stop()

target:  [2, 2]
gamma:  0 direction:  forward distance:  3
alpha:  0.0 direction:  forward
robot:  [2, 2]
target:  [3, 2]
gamma:  0 direction:  forward distance:  3
alpha:  0.0 direction:  forward
robot:  [3, 2]
target:  [4, 3]
gamma:  45.0 direction:  left distance:  4.242640687119285
alpha:  45.0 direction:  forward
robot:  [4, 3]
target:  [5, 4]
gamma:  0.0 direction:  left distance:  4.242640687119285
alpha:  45.0 direction:  forward
robot:  [5, 4]
target:  [5, 5]
gamma:  45.0 direction:  left distance:  3
alpha:  90.0 direction:  forward
robot:  [5, 5]
